## **Airline Flight Statistics**

In [1]:
pip install dash

     |████████████████████████████████| 7.3 MB 4.8 MB/s 
     |████████████████████████████████| 25.3 MB 68 kB/s 
     |████████████████████████████████| 357 kB 53.7 MB/s 
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3821 sha256=214ec03fa038e4f0739301f102ec27a7151858d57e5207d5b0e35974c6a3e700
  Stored in directory: /root/.cache/pip/wheels/00/f9/c7/1a6437d794ed753ea9bc9079e761d4fc803a1f1f5d3697b9ec
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4089 sha256=57f2f003590bb26b28e0128ec95081bc937cf9c425f9e031a91d8459509145bf
  Stored in directory: /root/.cache/pip/wheels/ec/6b/81/05aceabd8b27f724e2c96784016287cc1bfbc349ebfda451de
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl size=3911 sha256=8e61bf508914519da41772c07ede5af0b27eff92bdb3b264da8f6216652b5ed6
  Stored in directory: /root/.cache/pip/wheels/85/5d/4e/7c276b57992951dbe770bf5caad6448d0539c510663aefd

In [2]:
pip install jupyter_dash

In [3]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px
import numpy as np


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  after removing the cwd from sys.path.


In [4]:
JupyterDash.infer_jupyter_proxy_config()

In [5]:
JupyterDash.infer_jupyter_proxy_config()

In [6]:
airline_data =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv', 
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str, 
                                   'Div2Airport': str, 'Div2TailNum': str})

In [7]:
airline_data['SecurityDelay']=airline_data['SecurityDelay'].fillna(0)

In [8]:
airline_data.head()

,Unnamed: 0,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,...,Div1Airport,Div1AirportID,Div1AirportSeqID,Div1WheelsOn,Div1TotalGTime,Div1LongestGTime,Div1WheelsOff,Div1TailNum,Div2Airport,Div2AirportID,Div2AirportSeqID,Div2WheelsOn,Div2TotalGTime,Div2LongestGTime,Div2WheelsOff,Div2TailNum,Div3Airport,Div3AirportID,Div3AirportSeqID,Div3WheelsOn,Div3TotalGTime,Div3LongestGTime,Div3WheelsOff,Div3TailNum,Div4Airport,Div4AirportID,Div4AirportSeqID,Div4WheelsOn,Div4TotalGTime,Div4LongestGTime,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum
0,1295781,1998,2,4,2,4,1998-04-02,AS,19930,AS,N785AS,584,11884,1188401,31884,GEG,"Spokane, WA",WA,53.0,Washington,93,14747,1474702,30559,SEA,"Seattle, WA",WA,53.0,Washington,93,1330,1330.0,0.0,0.0,0.0,0.0,1300-1359,8.0,1338.0,1415.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1125375,2013,2,5,13,1,2013-05-13,EV,20366,EV,N24103,4132,11618,1161802,31703,EWR,"Newark, NJ",NJ,34.0,New Jersey,21,14524,1452401,34524,RIC,"Richmond, VA",VA,51.0,Virginia,38,1301,1255.0,-6.0,0.0,0.0,-1.0,1300-1359,9.0,1304.0,1358.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,118824,1993,3,9,25,6,1993-09-25,UA,19977,UA,NaN,2206,14108,1410801,34108,PIA,"Peoria, IL",IL,17.0,Illinois,41,13930,1393001,30977,ORD,"Chicago, IL",IL,17.0,Illinois,41,1650,1723.0,33.0,33.0,1.0,2.0,1600-1659,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,634825,1994,4,11,12,6,1994-11-12,HP,19991,HP,NaN,1207,12892,1289201,32575,LAX,"Los Angeles, CA",CA,6.0,California,91,14107,1410701,30466,PHX,"Phoenix, AZ",AZ,4.0,Arizona,81,1245,1309.0,24.0,24.0,1.0,1.0,1200-1259,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1888125,2017,3,8,17,4,2017-08-17,UA,19977,UA,N827UA,576,11003,1100303,31003,CID,"Cedar Rapids/Iowa City, IA",IA,19.0,Iowa,61,11292,1129202,30325,DEN,"Denver, CO",CO,8.0,Colorado,82,755,746.0,-9.0,0.0,0.0,-1.0,0700-0759,8.0,754.0,836.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
colors = {
    'background': '#ADD8E6',
    'text': '#111111'
}

In [10]:

app = JupyterDash(__name__)

def compute_info(airline_data, entered_year):
    df =  airline_data[airline_data['Year']==int(entered_year)]
    
    avg_car = df.groupby(['Month','Reporting_Airline'])['CarrierDelay'].mean().reset_index()
    avg_weather = df.groupby(['Month','Reporting_Airline'])['WeatherDelay'].mean().reset_index()
    avg_NAS = df.groupby(['Month','Reporting_Airline'])['NASDelay'].mean().reset_index()
    avg_sec = df.groupby(['Month','Reporting_Airline'])['SecurityDelay'].mean().reset_index()
    avg_late = df.groupby(['Month','Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()
    return avg_car, avg_weather, avg_NAS, avg_sec, avg_late

app.layout=html.Div(style={'backgroundColor': colors['background']},children=[html.H1('Airline DashBoard',style={'textAlign':'center','color':'#111111','font-size':35}),
                    html.Div(['Input Year : ',dcc.Input(id='input-year',value=2010,type='number',style={'height':'50px','font-size':25})],style={'font-size':35}),
                    html.Br(),
                    html.Br(),
                    html.Div([
                    html.Div(dcc.Graph(id='carrier-plot'),style={'width':'50%'}),
                    html.Div(dcc.Graph(id='weather-plot'),style={'width':'50%'}),],
                    style={'display':'flex'}),
                    html.Div([
                    html.Div(dcc.Graph(id='nasdelay-plot'),style={'width':'50%'}),
                    html.Div(dcc.Graph(id='lateaircraft-plot'),style={'width':'50%'})
                    ,],
                    style={'display':'flex'}),
                    html.Div(dcc.Graph(id='security-plot'),style={'width':'50%'}),
                    
])

@app.callback([Output(component_id='carrier-plot',component_property='figure'),
               Output(component_id='weather-plot',component_property='figure'),
               Output(component_id='nasdelay-plot',component_property='figure'),
               Output(component_id='security-plot',component_property='figure'),
               Output(component_id='lateaircraft-plot',component_property='figure')],
               Input(component_id='input-year',component_property='value')
               )

def get_graph(entered_year):
  
  avg_car, avg_weather, avg_NAS, avg_sec, avg_late=compute_info(airline_data,entered_year)

  fig_avg_car=px.bar(avg_car,x='Month',y='CarrierDelay',title='Carrier Delay time by Airline',color='Reporting_Airline',)
  fig_avg_weather=px.bar(avg_weather,x='Month',y='WeatherDelay',title='WeatherDelay time by Airline',color='Reporting_Airline')
  fig_avg_nas=px.bar(avg_NAS,x='Month',y='NASDelay',title='NAS Delay time by Airline',color='Reporting_Airline')
  fig_avg_sec=px.bar(avg_sec,x='Month',y='SecurityDelay',title='Security Delay time by Airline',color='Reporting_Airline')
  fig_avg_late=px.bar(avg_late,x='Month',y='LateAircraftDelay',title='Lat eAircraft Delay time by Airline',color='Reporting_Airline')

  return [fig_avg_car,fig_avg_weather,fig_avg_nas,fig_avg_sec,fig_avg_late]

    
    
if __name__=='__main__':
  app.run_server(host='127.0.0.1',port='8000',debug=True)



Dash app running on:


<IPython.core.display.Javascript object>